In [18]:
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit
from scipy.stats import norm, stats
from os import path, listdir
from datetime import datetime



In [47]:
#ARAM CLOCK CODE
def add_clock(tree):
    channel = tree['channel'].array()
    time = tree['time'].array()
    nSamples = len(time)
    clocks = channel[:,2] # Hardcoded clock channel
    triggers = channel[:,1] # Hardcoded trigger channel
    times = np.array(time[:,0])*10**9
    # breakpoint()
    clock = np.array(clocks)
    minima = np.tile(np.min(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    maxima = np.tile(np.max(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    amp_fraction = 20 # %
    amp = minima + np.abs(minima - maxima)*amp_fraction/100
    min_scale = np.abs(maxima - minima)/10.0
    clock_diff = np.diff(clock, append=0)
    clock_diff_mask = clock_diff > min_scale
    # true after indices
    check_prior_fall = clock_diff < -min_scale
    prior_indices = np.argmax(check_prior_fall, axis=1)
    prior_fall_mask = np.arange(check_prior_fall.shape[1]) >= prior_indices[:, None]
    # breakpoint()
    global_mask = clock_diff_mask & prior_fall_mask
    # breakpoint()
    times = np.where(global_mask, times, 0)
    clock = np.where(global_mask, clock, 0)
    # delete 0 values for each row
    # breakpoint()
    times = ak.Array([sublist[sublist != 0] for sublist in times])
    clock = ak.Array([sublist[sublist != 0] for sublist in clock])
    #print(times)
    # breakpoint()
    time_slope = times[:,1] - times[:,0] #x run
    clock_slope = clock[:,1] - clock[:,0] #y rise
    slope = clock_slope / time_slope #y rise / x run
    ybias = clock[:,0] - slope*times[:,0]
    # calculate 20% of the amplitude
    amp = (minima + np.abs(minima - maxima)*amp_fraction/100)[:,0]
    clock_timestamp = np.array((amp - ybias) / slope)
    return times, clock, clock_slope, time_slope, slope, clock_timestamp


def custom_clock(events, amp_fraction=20, plot_num=None):
    import matplotlib.pyplot as plt
    def two_pnt_fit(times, clocks, edg_idx, amp):
        frst_times = times[:,edg_idx,0]
        scnd_times = times[:,edg_idx,1]

        frst_clocks = clocks[:,edg_idx,0]
        scnd_clocks = clocks[:,edg_idx,1]

        t_run = scnd_times - frst_times
        c_rise = scnd_clocks - frst_clocks
        slope = c_rise / t_run
        ybias = frst_clocks - slope*frst_times
        # calculate 20% of the amplitude
        #amp = (minima + np.abs(minima - maxima)*amp_fraction/100)[:,0]
        c_timestamp = np.array((amp[:,0] - ybias) / slope)
        return c_timestamp

    channel = events['channel']
    time = events['time']
    clocks = channel[:,2] # Hardcoded clock channel
    times = np.array(time[:,0])*10**9
    clock = np.array(clocks)

    if plot_num is not None:
        fig, ax = plt.subplots(1,1, figsize=(16,8))
        ax.scatter(times[plot_num], clock[plot_num], color='lightgrey')

    minima = np.tile(np.min(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    maxima = np.tile(np.max(clock, axis=1).reshape(-1,1), (1, len(clock[0])))
    amp = minima + np.abs(minima - maxima)*amp_fraction/100

    min_scale = np.abs(maxima - minima)/10.0

    clock_diff = np.diff(clock, append=0)
    rising_edge_mask = clock_diff > min_scale #to get all rising edges min_scale is a positive number

    all_rising_times = np.where(rising_edge_mask, times, 0)
    all_rising_clocks = np.where(rising_edge_mask, clock, 0)
    all_rising_times = ak.Array([sublist[sublist != 0] for sublist in all_rising_times])
    all_rising_clocks = ak.Array([sublist[sublist != 0] for sublist in all_rising_clocks])

    t_diffs = [np.diff(t_e) for t_e in all_rising_times]
    cut_points = [np.where(diff > 5)[0]+1 for diff in t_diffs]
    
    split_times = [np.split(t_e, ct_p) for t_e, ct_p in zip(all_rising_times, cut_points)]
    split_clocks = [np.split(t_e, ct_p) for t_e, ct_p in zip(all_rising_clocks, cut_points)]
    
    nmask = ak.num(split_times, axis=2) > 2 #keep only edges with more than 2 points point, doing keeping one than mask on biggest points is best practice lets try this tho

    rising_times = ak.Array(split_times)[nmask]
    rising_clocks = ak.Array(split_clocks)[nmask]

    #one could maybe do another mask that only keeps two arrays with the most points

    if plot_num is not None:
        ax.hlines(amp[plot_num], xmin=-25, xmax=25,color='r', linestyles="--", label="amp")
        ax.hlines(minima[plot_num], xmin=-25, xmax=25,color='maroon', linestyles="--", label="minima")
        ax.hlines(maxima[plot_num], xmin=-25, xmax=25,color='maroon', linestyles="--", label="maxima")

        ax.scatter(all_rising_times[plot_num], all_rising_clocks[plot_num], color='blue')
        ax.scatter(rising_times[plot_num][0], rising_clocks[plot_num][0], color='cyan', label="firt rising edge")
        ax.scatter(rising_times[plot_num][1], rising_clocks[plot_num][1], color='deeppink', label="second rising edge")
        ax.set_xlabel("time (ns)")
        ax.set_ylabel("Clock Voltage")
        ax.legend(loc=1, prop={'size': 12}, frameon=True,fancybox=True)
    
    clock_edge_0 = two_pnt_fit(rising_times, rising_clocks, 0, amp)
    clock_edge_1 = two_pnt_fit(rising_times, rising_clocks, 1, amp)

    Ts = clock_edge_1 - clock_edge_0

    clk_nearest_zero = np.where((np.abs(clock_edge_0) <= np.abs(clock_edge_1)), clock_edge_0, clock_edge_1 )

    return Ts, clock_edge_0, clock_edge_1, clk_nearest_zero

In [ ]:
#This cell checks to see if any clock fits fail using Arams old Clock code
data_path = '/ceph/cms/store/user/iareed/ETL/Desy_Mar24'
run_file = 'run_5051.root'

run_data = uproot.open( path.join(data_path, run_file) )['pulse']


fig, axs = plt.subplots(1, 2, sharey=False, tight_layout=True)
n_bins = 50

times, clocks, dclocks, dtimes, slopes, clock_timestamps = add_clock(run_data)

# large_tdiff_sel = dtimes < -0.25
print(ak.count_nonzero(dtimes < 0), ak.max(dtimes))
# print(dtimes[large_tdiff_sel])

axs[0].hist(dclocks, bins=n_bins, color='b')
#axs[1].hist(dtimes, bins=np.linspace(-0.1,0.15,50), color='b')

my_bins = np.linspace(-0.1,0.2,50)
axs[1].hist(np.clip(dtimes, my_bins[0], my_bins[-1]), bins=my_bins)

# fig, ax = plt.subplots(1,1, figsize=(16,8))
# ax.scatter(times[plot_num], clock[plot_num], color='black')

axs[0].set_xlabel('Amplitude Difference')
axs[0].set_ylabel('Counts')

axs[1].set_xlabel('Time Difference')
plt.show()